Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [10]:
import numpy as np, tensorflow as tf, pickle

In [2]:
pickle_file = '/Volumes/SamuiPassport/tmp/notMNIST.pickle'


with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.132631
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.931578
Minibatch accuracy: 18.8%
Validation accuracy: 40.4%
Minibatch loss at step 100: 1.147332
Minibatch accuracy: 62.5%
Validation accuracy: 68.1%
Minibatch loss at step 150: 0.438605
Minibatch accuracy: 87.5%
Validation accuracy: 74.4%
Minibatch loss at step 200: 0.701477
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 250: 1.223293
Minibatch accuracy: 68.8%
Validation accuracy: 77.7%
Minibatch loss at step 300: 0.420621
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.361625
Minibatch accuracy: 93.8%
Validation accuracy: 77.7%
Minibatch loss at step 400: 0.359544
Minibatch accuracy: 93.8%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.825923
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.766928
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [181]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        maxpool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], 'SAME')
        conv = tf.nn.conv2d(maxpool, layer2_weights, [1,1,1,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        maxpool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], 'SAME')
        shape = maxpool.get_shape().as_list()
        reshape = tf.reshape(maxpool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [182]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.026077
Minibatch accuracy: 0.0%
Validation accuracy: 9.0%
Minibatch loss at step 50: 1.877824
Minibatch accuracy: 18.8%
Validation accuracy: 43.1%
Minibatch loss at step 100: 0.992196
Minibatch accuracy: 50.0%
Validation accuracy: 65.6%
Minibatch loss at step 150: 0.523380
Minibatch accuracy: 81.2%
Validation accuracy: 72.5%
Minibatch loss at step 200: 0.891117
Minibatch accuracy: 75.0%
Validation accuracy: 76.7%
Minibatch loss at step 250: 1.369560
Minibatch accuracy: 62.5%
Validation accuracy: 78.5%
Minibatch loss at step 300: 0.347681
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.581528
Minibatch accuracy: 93.8%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.198531
Minibatch accuracy: 100.0%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.801777
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500: 0.627634
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Mi

### My comments on the above
Adding max_pooling increased test accuracy from 89.6% to 90.9%. On the 10k test set the 0.13% is 13 cases better. Yet this is way worse than the simple deep learning model from Assignment3 where I gained 95.6% accuracy on test set.

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [246]:
batch_size = 32
patch_size = 5
depth = 16
num_hidden = 64
seed=4242

tf.reset_default_graph()

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    

    def conv_relu_maxpool(inputs, kernel_shape, bias_shape, bias_init_v=0.0):
        weights = tf.get_variable("weights", shape = kernel_shape,
                                 initializer = tf.truncated_normal_initializer(stddev=0.1, seed=seed))
        biases = tf.get_variable("biases", shape = bias_shape,
                                initializer = tf.constant_initializer(bias_init_v))
        conv = tf.nn.conv2d(inputs, weights, [1,1,1,1], padding='SAME')
        hidden = tf.nn.relu(conv + biases)
        maxpool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], 'SAME')
        
        return maxpool

    
    def dense(inputs, units, activation=None):
        return tf.layers.dense(
            inputs,
            units=units,
            activation=activation,
            kernel_initializer=tf.truncated_normal_initializer(stddev=0.05, seed=seed),
            bias_initializer=tf.constant_initializer(0.1),
            name="Dense"
        )

    
    # Model.
    def model(data, keep_prob=1):
        with tf.variable_scope("conv1"):
            layer1 = conv_relu_maxpool(data, [patch_size, patch_size, num_channels, depth], [depth], 0.01)
            
        with tf.variable_scope("conv2"):
            layer2 = conv_relu_maxpool(layer1, [patch_size, patch_size, depth, depth], [depth], 1.0)
        shape = layer2.get_shape().as_list()
        reshape = tf.reshape(layer2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        with tf.variable_scope("dense1"):
            dense1 = dense( reshape, 64, activation=tf.nn.relu)
        dropout1 = tf.nn.dropout( dense1, keep_prob=keep_prob, seed=seed)
        with tf.variable_scope("dense2"):
            dense2 = dense( dense1, 32, activation=tf.nn.relu)
        dropout2 = tf.nn.dropout( dense2, keep_prob=keep_prob, seed=seed)
        with tf.variable_scope("dense3"):
            dense3 = dense( dropout2, num_labels)
            
        return dense3

    
    with tf.variable_scope("model"):
        # Training computation.
        logits = model(tf_train_dataset, keep_prob=0.7)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

        # Optimizer.
        global_step = tf.Variable(0)  # count the number of steps taken.
        learning_rate = tf.train.exponential_decay(0.05, global_step, 1001, 0.8)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)

    with tf.variable_scope("model", reuse=True):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
        
    with tf.variable_scope("model", reuse=True):
        test_prediction = tf.nn.softmax(model(tf_test_dataset))


Type: <class 'tensorflow.python.framework.ops.Tensor'>
Type: <class 'tensorflow.python.framework.ops.Tensor'>
Type: <class 'tensorflow.python.framework.ops.Tensor'>


In [212]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Step {:4d} rate {:.4f} '.format(step, learning_rate.eval()), end='')
            accPred = accuracy(predictions, batch_labels)
            accValid = accuracy(valid_prediction.eval(), valid_labels)
            print('Loss {:2.3f}  Batch acc.: {:02.2f}%  Validation acc.: {:2.2f}%'.format(l,accPred,accValid))
            
    print('\nTest accuracy: {:.2f}%'.format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Step    0 rate 0.0500 Loss 2.334  Batch acc.: 0.00%  Validation acc.: 10.31%
Step   50 rate 0.0494 Loss 2.307  Batch acc.: 3.12%  Validation acc.: 9.72%
Step  100 rate 0.0489 Loss 2.263  Batch acc.: 12.50%  Validation acc.: 12.58%
Step  150 rate 0.0483 Loss 2.057  Batch acc.: 28.12%  Validation acc.: 26.38%
Step  200 rate 0.0478 Loss 1.455  Batch acc.: 53.12%  Validation acc.: 41.73%
Step  250 rate 0.0473 Loss 1.325  Batch acc.: 56.25%  Validation acc.: 56.86%
Step  300 rate 0.0468 Loss 0.956  Batch acc.: 68.75%  Validation acc.: 66.23%
Step  350 rate 0.0462 Loss 1.052  Batch acc.: 62.50%  Validation acc.: 69.31%
Step  400 rate 0.0457 Loss 0.830  Batch acc.: 71.88%  Validation acc.: 74.87%
Step  450 rate 0.0452 Loss 1.304  Batch acc.: 68.75%  Validation acc.: 75.71%
Step  500 rate 0.0447 Loss 0.756  Batch acc.: 75.00%  Validation acc.: 78.38%
Step  550 rate 0.0442 Loss 0.846  Batch acc.: 75.00%  Validation acc.: 80.26%
Step  600 rate 0.0437 Loss 1.022  Batch acc.: 71.88%  V

### tf.layers ftw

In [7]:
batch_size = 32
patch_size = 5
depth = 16
num_hidden = 64
seed=4242

graph = tf.Graph()
tf.reset_default_graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    # Model.
    def model(data, keep_prob=0.5, training=False):
        conv_1 = tf.layers.conv2d(
            data, filters=depth, kernel_size=1, padding='SAME',
            activation=tf.nn.relu,
            kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=seed),
            bias_initializer=tf.constant_initializer(0.0),
            name="Conv_1"
        )
        pool_1 = tf.layers.max_pooling2d( conv_1, pool_size=patch_size, strides=2, padding='SAME')

        conv_2 = tf.layers.conv2d(
            pool_1, filters=depth, kernel_size=1, padding='SAME',
            activation=tf.nn.relu,
            kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=2*seed),
            bias_initializer=tf.constant_initializer(1.0),
            name="Conv_2"
        )
        pool_2 = tf.layers.max_pooling2d( conv_2, pool_size=patch_size, strides=2, padding='SAME')

        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        dense_1 = tf.layers.dense(
            reshape, units=num_hidden, activation=tf.nn.relu,
            kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=seed),
            bias_initializer=tf.constant_initializer(1.0)
        )

        dropout_1 = tf.layers.dropout( dense_1, rate=(1.0-keep_prob), seed=seed, training=training)

        dense_2 = tf.layers.dense(
            dropout_1, units=num_labels, activation=None,
            kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=seed),
            bias_initializer=tf.constant_initializer(1.0)
        )

        return dense_2

    
    with tf.variable_scope("model"):
        # Training computation.
        logits = model(tf_train_dataset, training=True)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

        # Optimizer.
        global_step = tf.Variable(0, name="globalStep", trainable=False)  # count the number of steps taken.
        learning_rate = tf.train.exponential_decay(0.05, global_step, 1001, 0.8)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)

    with tf.variable_scope("model", reuse=True):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
        
    with tf.variable_scope("model", reuse=True):
        test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Step {:4d} rate {:.4f} '.format(step, learning_rate.eval()), end='')
            accPred = accuracy(predictions, batch_labels)
            accValid = accuracy(valid_prediction.eval(), valid_labels)
            print('Loss {:2.3f}  Batch acc.: {:02.2f}%  Validation acc.: {:2.2f}%'.format(l,accPred,accValid))

    print('\nTest accuracy: {:.2f}%'.format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Step    0 rate 0.0500 Loss 4.903  Batch acc.: 18.75%  Validation acc.: 9.98%
Step   50 rate 0.0494 Loss 2.301  Batch acc.: 15.62%  Validation acc.: 10.00%
Step  100 rate 0.0489 Loss 2.302  Batch acc.: 12.50%  Validation acc.: 10.00%
Step  150 rate 0.0483 Loss 2.303  Batch acc.: 6.25%  Validation acc.: 10.00%
Step  200 rate 0.0478 Loss 2.306  Batch acc.: 0.00%  Validation acc.: 10.00%
Step  250 rate 0.0473 Loss 2.306  Batch acc.: 3.12%  Validation acc.: 10.00%
Step  300 rate 0.0468 Loss 2.305  Batch acc.: 15.62%  Validation acc.: 10.00%
Step  350 rate 0.0462 Loss 2.304  Batch acc.: 15.62%  Validation acc.: 10.00%
Step  400 rate 0.0457 Loss 2.307  Batch acc.: 6.25%  Validation acc.: 10.00%


KeyboardInterrupt: 